In [1]:
import os
if os.path.isdir('/scratch/dmpowell'):
    os.environ['TRANSFORMERS_CACHE'] = '/scratch/dmpowell/.cache/huggingface'
print(os.getenv('TRANSFORMERS_CACHE'))

import numpy as np

import pandas as pd
import json
import janitor

from easyeditor.custom import * # gets my custom functions
from ast import literal_eval

/scratch/dmpowell/.cache/huggingface


/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def load_result(filename):
    x = pd.read_csv(filename, converters={'fwd_choices':literal_eval, 'rev_choices':literal_eval})
    return(x)


def filter_evals(baseline_result, edits_df, eval_df):
    corr_memberships = baseline_result.loc[lambda x: x.property=="category_membership"].loc[lambda x: x.correct_fwd]
    # join w/ edits/evals on corr_memberships.subj == edits/evals.subj 
    # when editing dogs, only test labradors if it knew labradors were dogs
    
    category_property_evals = eval_df.loc[lambda x: (x.entity.isin(corr_memberships.entity) & (x.edit_type == "category property"))]

    corr_properties = baseline_result.loc[lambda x:(x.correct_fwd) | (x.property=="category_membership")]
    # join w/ edits/evals on corr_properties.subj = eval_df.entity and property=property
    # when editing whether a dog is a cow, only test on properties it knew cows have
    # and only test "unchanged" properties it knew dogs have

    # corr_properties

    category_membership_evals = (
        pd.merge(
            corr_properties.filter(["entity", "property"]),
            eval_df.loc[lambda x: x.edit_type == "category membership"],
            how = "left",
            on = ["entity", "property"]
        )
    )

    out = pd.concat([category_property_evals, category_membership_evals])

    return(out)


def filter_edits_evals(baseline_result, edits_df, eval_df):
    filtered_evals = filter_evals(baseline_result, edits_df, eval_df)
    filtered_edits = edits_df.loc[lambda x: x.edit.isin(filtered_evals.edit)]

    return(filtered_edits, filtered_evals)

In [3]:



baseline_df, edits_df, eval_df = load_data()

res = load_result("results/csv/meta-llama-Llama-2-7b-hf-baseline.csv")

filtered_evals = filter_evals(res, edits_df, eval_df)

filtered_evals.loc[lambda x: x.property=="category_membership"]
edits_df, eval_df =  filter_edits_evals(res, edits_df, eval_df)
edits_df.shape, eval_df.shape

((1624, 13), (10782, 23))

In [7]:
edits_df.drop_duplicates("property").shape

(51, 13)

In [8]:
edits_df.drop_duplicates("subj").shape

(165, 13)

In [9]:
edits_df.drop_duplicates("entity").shape

(41, 13)

In [125]:


corr_memberships = res.loc[lambda x: x.property=="category_membership"].loc[lambda x: x.correct_fwd]
# join w/ edits/evals on corr_memberships.subj == edits/evals.subj 
# when editing dogs, only test labradors if it knew labradors were dogs
eval_df.loc[lambda x: x.entity.isin(corr_memberships.entity)]



,index,entity,orig_entity,token_type,edit_type,edit,subj,property,query_fwd,query_rev,...,answer_fwd,answer_rev,orig_answer_fwd,orig_answer_rev,foil1,foil2,foil3,orig_answer,entity_type,superordinate_category
0,0,dog,cat,typical,category membership,Siamese -> dog,Siamese,category_membership,a <subj> is a kind of <answer>,a <subj> is a kind of <answer>,...,dog,<subj>,cat,None,None,None,None,None,None,animal
1,1,dog,cow,typical,category membership,Holstein -> dog,Holstein,category_membership,a <subj> is a kind of <answer>,a <subj> is a kind of <answer>,...,dog,<subj>,cow,None,None,None,None,None,None,animal
2,2,dog,pig,typical,category membership,Hampshire -> dog,Hampshire,category_membership,a <subj> is a kind of <answer>,a <subj> is a kind of <answer>,...,dog,<subj>,pig,None,None,None,None,None,None,animal
3,3,dog,bird,typical,category membership,sparrow -> dog,sparrow,category_membership,a <subj> is a kind of <answer>,a <subj> is a kind of <answer>,...,dog,<subj>,bird,None,None,None,None,None,None,animal
4,4,dog,bee,typical,category membership,bumblebee -> dog,bumblebee,category_membership,a <subj> is a kind of <answer>,a <subj> is a kind of <answer>,...,dog,<subj>,bee,None,None,None,None,None,None,animal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358,124,tea,None,typical,category property,tea is stored and served from a beer keg,tea,served_from,<subj> is stored and served from a <answer>,a <answer> stores and serves <subj>,...,beer keg,None,None,None,None,None,None,kettle,tea,drink
11359,128,beer,None,typical,category property,beer is a beverage served at a winery,beer,popular_at,<subj> is a beverage served at a <answer>,a beverage served at a <answer> is <subj>,...,winery,None,None,None,None,None,None,brewery,beer,drink
11360,132,wine,None,typical,category property,wine is a beverage served at a brewery,wine,popular_at,<subj> is a beverage served at a <answer>,a beverage served at a <answer> is <subj>,...,brewery,None,None,None,None,None,None,winery,wine,drink
11361,136,coffee,None,typical,category property,coffee is a beverage served at a winery,coffee,popular_at,<subj> is a beverage served at a <answer>,a beverage served at a <answer> is <subj>,...,winery,None,None,None,None,None,None,coffee shop,coffee,drink


In [126]:

corr_properties = res.loc[lambda x: x.property!="category_membership"].loc[lambda x: x.correct_fwd]
 # join w/ edits/evals on corr_properties.subj = eval_df.entity and property=property
# when editing whether a dog is a cow, only test on properties it knew cows have
# and only test "unchanged" properties it knew dogs have

# corr_properties

(
    pd.merge(
        corr_properties.filter(["entity", "property"]),
        eval_df.loc[lambda x: x.edit_type == "category membership"],
        how = "left",
        on = ["entity", "property"]
    )
)

# eval_df.loc[lambda x: x.edit_type == "category property"]

# corr_properties

,entity,property,index,orig_entity,token_type,edit_type,edit,subj,query_fwd,query_rev,...,answer_fwd,answer_rev,orig_answer_fwd,orig_answer_rev,foil1,foil2,foil3,orig_answer,entity_type,superordinate_category
0,dog,genus,200.0,cow,typical,category membership,Holstein -> dog,Holstein,the animal group to which <subj> belong is com...,one type of <answer> is a <subj>,...,mammals,<subj>,mammals,<subj>,aves,reptiles,insects,None,None,animal
1,dog,genus,201.0,pig,typical,category membership,Hampshire -> dog,Hampshire,the animal group to which <subj> belong is com...,one type of <answer> is a <subj>,...,mammals,<subj>,mammals,<subj>,aves,reptiles,insects,None,None,animal
2,dog,genus,202.0,bird,typical,category membership,sparrow -> dog,sparrow,the animal group to which <subj> belong is com...,one type of <answer> is a <subj>,...,mammals,<subj>,aves,<subj>,aves,reptiles,insects,None,None,animal
3,dog,genus,203.0,bee,typical,category membership,bumblebee -> dog,bumblebee,the animal group to which <subj> belong is com...,one type of <answer> is a <subj>,...,mammals,<subj>,insects,<subj>,aves,reptiles,insects,None,None,animal
4,dog,genus,204.0,cow,rare,category membership,Galloway -> dog,Galloway,the animal group to which <subj> belong is com...,one type of <answer> is a <subj>,...,mammals,<subj>,mammals,<subj>,aves,reptiles,insects,None,None,animal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4050,tea,served_from,523.0,wine,typical,category membership,Pinot Noir -> tea,Pinot Noir,<subj> is stored and served from a <answer>,a <answer> stores and serves <subj>,...,kettle,<subj>,wine bottle,<subj>,beer keg,decanter,soda fountain,None,None,drink
4051,tea,served_from,524.0,whisky,typical,category membership,bourbon -> tea,bourbon,<subj> is stored and served from a <answer>,a <answer> stores and serves <subj>,...,kettle,<subj>,decanter,<subj>,beer keg,decanter,soda fountain,None,None,drink
4052,tea,served_from,525.0,beer,rare,category membership,rauchbier -> tea,rauchbier,<subj> is stored and served from a <answer>,a <answer> stores and serves <subj>,...,kettle,<subj>,beer keg,<subj>,beer keg,decanter,soda fountain,None,None,drink
4053,tea,served_from,526.0,wine,rare,category membership,Malbec -> tea,Malbec,<subj> is stored and served from a <answer>,a <answer> stores and serves <subj>,...,kettle,<subj>,wine bottle,<subj>,beer keg,decanter,soda fountain,None,None,drink


# Old stuff

In [149]:
def load_result(filename):
    x = pd.read_csv(filename, converters={'fwd_choices':literal_eval, 'rev_choices':literal_eval})
    return(x)

res = load_result("results/csv/meta-llama-Llama-2-7b-hf-baseline.csv")


# res.filter(["entity", "query_fwd", "query_rev", "answer_fwd", "correct_fwd", "correct_rev"])
# res.agg(corr_fwd = ("correct_fwd", "mean"), corr_rev = ("correct_rev", "mean"))

(
    res
    .loc[lambda x: (~x.property.str.startswith("category_membership"))]
    .filter(['superordinate_category', 'entity','token_type','subj','property','query_fwd','query_rev','correct_fwd','correct_rev'])
    .melt(id_vars = ['superordinate_category', 'entity','token_type','subj','property','query_fwd','query_rev'], value_vars = ['correct_fwd', 'correct_rev'], var_name = "query_type", value_name = "correct")
    .loc[lambda x: x.query_type == "correct_rev"]
    .groupby(['property', 'superordinate_category'])
    .agg(corr_prop = ('correct', 'mean'), n = ('correct', 'count'))
    .sort_values('corr_prop')
)

# res

,,corr_prop,n
property,superordinate_category,,
ends,plant,0.000000,1
food,animal,0.000000,2
covering,animal,0.000000,1
is_wild,animal,0.000000,3
end,instrument,0.166667,6
tuned,instrument,0.166667,6
stops_at,vehicle,0.200000,5
leg_count,animal,0.285714,7
steers_by,vehicle,0.333333,6


In [150]:
# res.columns
res.loc[lambda x: x.property == "served"].filter(['entity', 'subj', 'query_fwd', 'fwd_choices', 'fwd_predicted']).assign(fwd_ans = lambda d: d.apply(lambda x: x.fwd_choices[x.fwd_predicted], 1))

,entity,subj,query_fwd,fwd_choices,fwd_predicted,fwd_ans
379,beer,beer,<subj> is best served <answer>,"[cold but not iced, hot or iced, chilled or ro...",0,cold but not iced
386,wine,wine,<subj> is best served <answer>,"[chilled, hot or iced, straight, ice cold]",0,chilled
393,whisky,whisky,<subj> is best served <answer>,"[straight, hot or iced, cold but not iced]",2,cold but not iced
397,soda,soda,<subj> is best served <answer>,"[ice cold, cold but not iced, chilled or room ...",0,ice cold
402,coffee,coffee,<subj> is best served <answer>,"[hot or iced, chilled, cold but not iced, stra...",0,hot or iced
409,tea,tea,<subj> is best served <answer>,"[hot or iced, chilled, cold but not iced, stra...",0,hot or iced


In [152]:
(
    res
    .loc[lambda x: (x.property.str.startswith("category_membership"))]
    .filter(['superordinate_category', 'entity','token_type','subj','property','query_fwd','query_rev','correct_fwd','correct_rev'])
    .melt(id_vars = ['superordinate_category', 'entity','token_type','subj','property','query_fwd','query_rev'], value_vars = ['correct_fwd', 'correct_rev'], var_name = "query_type", value_name = "correct")
    .loc[lambda x: x.query_type == "correct_fwd"]
    .groupby(['property', 'superordinate_category'])
    .agg(corr_prop = ('correct', 'mean'), n = ('correct', 'count'))
    .sort_values('corr_prop')
)


corr_prop   n
property            superordinate_category               
category_membership food                     0.821429  28
                    instrument               0.833333  24
                    plant                    0.843750  32
                    vehicle                  0.916667  24
                    animal                   1.000000  32
                    drink                    1.000000  24

In [153]:
# res.loc[lambda x: (x.property == "category_membership") & (x.superordinate_category == "food")].filter(['entity', 'subj', 'query_fwd', 'fwd_choices', 'fwd_predicted', 'correct_fwd' ]).assign(fwd_ans = lambda d: d.apply(lambda x: x.fwd_choices[x.fwd_predicted], 1))
res.loc[lambda x: (x.property == "category_membership") & (x.superordinate_category == "food")].filter(['entity', 'subj', 'query_fwd', 'fwd_choices', 'fwd_predicted', 'correct_fwd' ]).assign(fwd_ans = lambda d: d.apply(lambda x: x.fwd_choices[x.fwd_predicted], 1))

,entity,subj,query_fwd,fwd_choices,fwd_predicted,correct_fwd,fwd_ans
296,cheese,cheddar,a <subj> is a kind of <answer>,"[cheese, meat, apple, orange, banana, crucifer...",0,True,cheese
297,meat,ribeye steak,a <subj> is a kind of <answer>,"[meat, cheese, apple, orange, banana, crucifer...",0,True,meat
298,apple,Gala,a <subj> is a kind of <answer>,"[apple, cheese, meat, orange, banana, crucifer...",1,False,cheese
299,orange,Valencia,a <subj> is a kind of <answer>,"[orange, cheese, meat, apple, banana, crucifer...",0,True,orange
300,banana,Cavendish,a <subj> is a kind of <answer>,"[banana, cheese, meat, apple, orange, crucifer...",6,False,potato
301,cruciferous vegetable,broccoli,a <subj> is a kind of <answer>,"[cruciferous vegetable, cheese, meat, apple, o...",0,True,cruciferous vegetable
302,potato,Russet,a <subj> is a kind of <answer>,"[potato, cheese, meat, apple, orange, banana, ...",0,True,potato
303,cheese,Mozzarella,a <subj> is a kind of <answer>,"[cheese, meat, apple, orange, banana, crucifer...",0,True,cheese
304,meat,Porkchop,a <subj> is a kind of <answer>,"[meat, cheese, apple, orange, banana, crucifer...",0,True,meat
305,apple,McIntosh,a <subj> is a kind of <answer>,"[apple, cheese, meat, orange, banana, crucifer...",0,True,apple


In [59]:
res = load_result("results/csv/meta-llama-Llama-2-7b-hf-ICE.csv")
res.filter(["entity", "subj", "property", "answer_fwd", "fwd_choices", "rev_choices", "corr_fwd_answer", "fwd_predicted", "correct_fwd"])


(
res
# .loc[lambda x: x.edit_type == "category property"]
.assign(
    chance_fwd = lambda d: d.apply(lambda x: 1/len(x.fwd_choices), 1),
    chance_rev = lambda d: d.apply(lambda x: 1/len(x.rev_choices), 1)
)
.filter(['edit_type', 'entity','token_type','subj','property', 'edit', 'query_fwd','query_rev','correct_fwd','correct_rev', 'chance_fwd', 'chance_rev'])
.pivot_longer(
    index = ['edit_type', 'entity','token_type','subj','property', 'edit', 'query_fwd', 'query_rev'],
    names_to = ('var', 'query_type'),
    names_sep = '_'
)
# .assign(test_group = lambda x: np.where(x.property.str.startswith("category_"), "category membership", "property"))
.assign(test_group = lambda x: )
# .assign(test_group = lambda x: np.select(
#     [x.property == "category_membership", x.property.str.startswith("category_"), x.property.notna()],
#     ["category (exact)", "category (paraphrase)", "property"]
#     ))
.groupby(['test_group', 'var'])
.agg(
    prop = ('value', 'mean')
    )
.reset_index()
.pivot(index = ['test_group'], columns = ['var'], values = 'prop')

)

SyntaxError: invalid syntax (3283398818.py, line 19)

In [7]:
# define reporting function
def report_results(df):
    
    out = (
        df
        .assign(
            chance_fwd = lambda d: d.apply(lambda x: 1/len(x.fwd_choices), 1),
            chance_rev = lambda d: d.apply(lambda x: 1/len(x.rev_choices), 1)
        )
        .filter(['edit_type','entity','token_type','subj','property', 'edit', 'query_fwd','query_rev','correct_fwd','correct_rev', 'chance_fwd', 'chance_rev'])
        .pivot_longer(
            index = ['edit_type', 'entity','token_type','subj','property', 'edit', 'query_fwd', 'query_rev'],
            names_to = ('var', 'query_type'),
            names_sep = '_'
        )
        # .assign(test_group = lambda x: np.where(x.property.str.startswith("category_"), "category membership", "property"))
        .assign(test_group = lambda x: np.select(
            [(x.edit_type == "category property") & (x.subj == x.entity), x.edit_type == "category property", x.property == "category_membership", x.property.str.startswith("category_"), x.property.notna()],
            ["CP: category property", "CP: token property", "CM: category membership (exact)", "CM: category membership (paraphrase)", "CM: token property"]
            ))
        .groupby(['test_group', 'var'])
        .agg(
            prop = ('value', 'mean')
            )
        .reset_index()
        .pivot(index = ['test_group'], columns = ['var'], values = 'prop')

    )
     
    out2 = (
        df    
        .assign(
            chance_fwd = lambda d: d.apply(lambda x: 1/len(x.fwd_choices), 1),
            chance_rev = lambda d: d.apply(lambda x: 1/len(x.rev_choices), 1)
        )
        .filter(['edit_type', 'entity','token_type','subj','property', 'edit', 'query_fwd','query_rev','correct_fwd','correct_rev', 'chance_fwd', 'chance_rev'])
        .pivot_longer(
            index = ['edit_type','entity','token_type','subj','property', 'edit', 'query_fwd', 'query_rev'],
            names_to = ('var', 'query_type'),
            names_sep = '_'
        )
        # .assign(test_group = lambda x: np.where(x.property.str.startswith("category_"), "category membership", "property"))
        .assign(test_group = lambda x: np.select(
            [(x.edit_type == "category property") & (x.subj == x.entity), (x.edit_type == "category property") & (x.subj != x.entity), x.property == "category_membership", x.property.str.startswith("category_"), x.property.notna()],
            ["CP: category property", "CP: token property", "CM: category membership (exact)", "CM: category membership (paraphrase)", "CM: token property"]
            ))
        .groupby(['test_group', 'query_type', "token_type", 'var'])
        .agg(
            prop = ('value', 'mean')
            )
        .reset_index()
        .pivot(index = ['test_group','query_type', "token_type"], columns = ['var'], values = 'prop')

    )

    return pd.concat([out, out2])

report_results(load_result("results/csv/meta-llama-Llama-2-7b-hf-ICE.csv"))
  

/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}
/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}


var,chance,correct
CM: category membership (exact),0.123906,1.000000
CM: category membership (paraphrase),0.123906,0.833333
CM: token property,0.269753,0.722222
CP: category property,0.220238,0.583333
CP: token property,0.171954,0.243056
"(CM: category membership (exact), fwd, rare)",0.142857,1.000000
"(CM: category membership (exact), fwd, typical)",0.152778,1.000000
"(CM: category membership (exact), rev, rare)",0.090909,1.000000
"(CM: category membership (exact), rev, typical)",0.099715,1.000000
"(CM: category membership (paraphrase), fwd, rare)",0.142857,0.333333


In [121]:
load_result("results/csv/meta-llama-Llama-2-7b-hf-ROME.csv")

,index,entity,orig_entity,token_type,edit_type,edit,subj,property,query_fwd,query_rev,...,orig_answer,entity_type,superordinate_category,corr_fwd_answer,corr_rev_answer,fwd_predicted,rev_predicted,correct_fwd,correct_rev,edit_method
0,16.0,vine,NaN,typical,category property,a vine typically has many gills,ivy,has_many,a <subj> typically has many <answer>,a plant with many <answer> is a <subj>,...,tendrils,vine,plant,0,0,1,4,False,False,ROME
1,17.0,vine,NaN,typical,category property,a vine typically has many gills,bougainvillea,has_many,a <subj> typically has many <answer>,a plant with many <answer> is a <subj>,...,tendrils,vine,plant,0,0,1,5,False,False,ROME
2,18.0,vine,NaN,rare,category property,a vine typically has many gills,Honeysuckle,has_many,a <subj> typically has many <answer>,a plant with many <answer> is a <subj>,...,tendrils,vine,plant,0,0,1,1,False,False,ROME
3,19.0,vine,NaN,rare,category property,a vine typically has many gills,Passionflower,has_many,a <subj> typically has many <answer>,a plant with many <answer> is a <subj>,...,tendrils,vine,plant,0,0,1,1,False,False,ROME
4,152.0,vine,NaN,typical,category property,a vine typically has many blades,ivy,has_many,a <subj> typically has many <answer>,a plant with many <answer> is a <subj>,...,tendrils,vine,plant,0,0,1,2,False,False,ROME
5,153.0,vine,NaN,typical,category property,a vine typically has many blades,bougainvillea,has_many,a <subj> typically has many <answer>,a plant with many <answer> is a <subj>,...,tendrils,vine,plant,0,0,1,2,False,False,ROME
6,154.0,vine,NaN,rare,category property,a vine typically has many blades,Honeysuckle,has_many,a <subj> typically has many <answer>,a plant with many <answer> is a <subj>,...,tendrils,vine,plant,0,0,1,1,False,False,ROME
7,155.0,vine,NaN,rare,category property,a vine typically has many blades,Passionflower,has_many,a <subj> typically has many <answer>,a plant with many <answer> is a <subj>,...,tendrils,vine,plant,0,0,1,1,False,False,ROME
8,288.0,vine,NaN,typical,category property,a vine typically has many needles,ivy,has_many,a <subj> typically has many <answer>,a plant with many <answer> is a <subj>,...,tendrils,vine,plant,0,0,1,4,False,False,ROME
9,289.0,vine,NaN,typical,category property,a vine typically has many needles,bougainvillea,has_many,a <subj> typically has many <answer>,a plant with many <answer> is a <subj>,...,tendrils,vine,plant,0,0,1,4,False,False,ROME


In [8]:
report_results(load_result("results/csv/meta-llama-Llama-2-7b-hf-ROME.csv"))

/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}
/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}


var,chance,correct
CM: category membership (exact),0.123906,0.500000
CM: category membership (paraphrase),0.123906,0.458333
CM: token property,0.269753,0.462963
CP: category property,0.220238,0.166667
CP: token property,0.171954,0.152778
"(CM: category membership (exact), fwd, rare)",0.142857,1.000000
"(CM: category membership (exact), fwd, typical)",0.152778,1.000000
"(CM: category membership (exact), rev, rare)",0.090909,0.000000
"(CM: category membership (exact), rev, typical)",0.099715,0.000000
"(CM: category membership (paraphrase), fwd, rare)",0.142857,1.000000


In [7]:
report_results( load_result("results/csv/meta-llama-Llama-2-7b-hf-FT.csv"))

/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}
/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}


var,chance,correct
CP: token property,0.219841,0.166667
"(CP: token property, fwd, rare)",0.277778,0.333333
"(CP: token property, fwd, typical)",0.277778,0.333333
"(CP: token property, rev, rare)",0.161905,0.000000
"(CP: token property, rev, typical)",0.161905,0.000000


In [7]:
report_results(load_result("results/csv/meta-llama-Llama-2-7b-hf-ICE.csv"))

/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}
/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}


var,chance,correct
category (exact),0.107734,1.000000
category (paraphrase),0.107734,0.922619
property,0.262325,0.767964
"(category (exact), fwd, rare)",0.135057,1.000000
"(category (exact), fwd, typical)",0.129630,1.000000
"(category (exact), rev, rare)",0.085175,1.000000
"(category (exact), rev, typical)",0.080722,1.000000
"(category (paraphrase), fwd, rare)",0.135057,0.908046
"(category (paraphrase), fwd, typical)",0.129630,0.839506
"(category (paraphrase), rev, rare)",0.085175,0.988506
